In [1]:
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm as tq
import matplotlib.pyplot as plt
from skimage.io import imshow
from pathlib import Path
import numpy as np
import random
import wandb

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import TensorDataset, DataLoader, Dataset

c:\Users\thoma\.conda\envs\deepL\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class MyEnsemble(nn.Module):
    def __init__(self, modelA, modelB, nb_classes=1):
        super(MyEnsemble, self).__init__()
        self.modelA = modelA
        self.modelB = modelB
        # Remove last linear layer
        self.modelA.out = nn.Identity()
        self.modelB.out = nn.Identity()
        
        # Create new classifier
        self.classifier = nn.Linear(18+18, nb_classes)
        
    def forward(self, x_l,x_r):
        x1 = self.modelA(x_l,x_r)  # clone to make sure x is not changed by inplace methods
        #x1 = x1.view(x1.size(0), -1)
        x2 = self.modelB(x_l,x_r)
        #x2 = x2.view(x2.size(0), -1)
        x = torch.cat((x1, x2), dim=1)
        
        x = self.classifier(F.relu(x))
        return x


In [3]:
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.apply(self._init_weights)
        self.conv = nn.Sequential(
            nn.Conv2d(3, 5, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(5, 5, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(5,7,3,padding=1),
            nn.ReLU(),
            nn.Flatten(),
        )

        
        self.fc1=nn.Sequential(
            nn.Linear(2268,18),
            nn.Sigmoid())

        self.out = nn.Linear(18,1)
        

    def _init_weights(self, module):
        if isinstance(module, nn.Conv2d):
            module.weight.data.normal_(mean=0.0, std=0.01)
            if module.bias is not None:
                module.bias.data.normal_(mean=0.5, std=0.01)
        if isinstance(module, nn.Linear):
            module.weight.data.normal_(mean=0.0, std=0.2)
            if module.bias is not None:
                module.bias.data.normal_(mean=0.5, std=0.01)
        
    def forward_once(self,inp):
        inp=self.conv(inp)
        inp=self.fc1(inp)
        return inp

    def forward(self, inp1, inp2):
        out1=self.forward_once(inp1)
        out2=self.forward_once(inp2)
        dis=torch.abs(out2-out1)
        out=self.out(dis)
        return out

In [4]:
# We use pretrained torchvision models here
modelA = torch.load('../data/net_87',map_location=torch.device('cpu'))
modelB = torch.load('../data/net_89',map_location=torch.device('cpu'))

In [5]:
# Train your separate models
# ...


# Freeze these models
for param in modelA.parameters():
    param.requires_grad_(False)

for param in modelB.parameters():
    param.requires_grad_(False)

# Create ensemble model
model = MyEnsemble(modelA, modelB)
x_l = torch.randn(1, 3, 75, 75)
x_r = torch.randn(1, 3, 75, 75)
output = model(x_l,x_r)

In [6]:
#/content/drive/MyDrive/Colab Notebooks/efficent net/
npz = np.load('../data/input_data.npz')
X_train = npz['X_train']
Y_train = npz['Y_train']

del npz

print('We have {} examples to work with'.format(Y_train.shape[0]))

We have 4113 examples to work with


In [7]:
class CustomImageDataset(Dataset):
    def __init__(self, left_list, right_list,targets, transform):
        self.left_dat = left_list
        self.right_dat = right_list
        self.targets = targets
        self.transform = transform
        self.imag_nomr = transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
        


    def __len__(self):
        return len(self.targets)

    def __getitem__(self, idx):
        left_img = self.left_dat[idx]
        right_img = self.right_dat[idx]
        targets = self.targets[idx]
        
        left_img = np.transpose(left_img,(2,0,1))
        left_img = torch.Tensor(left_img)

        right_img = np.transpose(right_img,(2,0,1))
        right_img = torch.Tensor(right_img)

        # nomalization of immages
        left_img = self.imag_nomr(left_img)
        right_img = self.imag_nomr(right_img)
                        
        if self.transform:
            left_img = self.transform(left_img)
            right_img = self.transform(right_img)
    
        return left_img, right_img, targets


def crate_Pairs_data(image_list,label_list, pairs = 5):
    """

    """
    left_input = []
    right_input = []
    targets = []

    #Number of pairs per image
    pairs = pairs
    #Let's create the new dataset to train on
    for i in range(len(label_list)):
        for _ in range(pairs): 
            # compare the same immage on the left to different immages to the right
            compare_to = i 
            while compare_to == i: #Make sure it's not comparing to itself
                compare_to = random.randint(0,len(image_list)-1)

            left_img = image_list[i][0]
            right_img = image_list[compare_to][0]

            # create data sets
            left_input.append(np.array(left_img))
            right_input.append(np.array(right_img))

            if label_list[i] == label_list[compare_to]:# They are the same
                targets.append(0.)
            else:# Not the same
                targets.append(1.)
    
    return left_input,right_input,targets

In [8]:
X_index, y_index = range(X_train.shape[0]), range(Y_train.shape[0])
# test 20% 
X_train_index, X_test_index, y_train_index, y_test_index = train_test_split(X_index, y_index,
    test_size=0.2, shuffle = True, random_state = 42)

# validation 20% training 60%
X_train_index, X_val_index, y_train_index, y_val_index = train_test_split(X_train_index, y_train_index, 
    test_size=0.20, random_state= 42) # 0.25 x 0.8 = 0.2

In [9]:
image_list_test = np.split(X_train[X_test_index],len(X_test_index))
label_list_test = np.split(Y_train[y_test_index],len(y_test_index))
left_dat,right_dat,targets = crate_Pairs_data(image_list = image_list_test,label_list=label_list_test,pairs=5)
test_dataset = CustomImageDataset(left_list=left_dat,right_list=right_dat,targets=targets,transform=None)

test_dataset_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1,shuffle=False, num_workers=2)

del image_list_test,label_list_test,test_dataset

In [10]:
def correct_out(output,label,TrueNegative,FalseNegative,TruePositive,FalsePositive):
    for j in range(output.size(0)):
        if (output[j]>0.5) and (label[j]==1):
            TrueNegative+=1
        elif (output[j]>0.5) and (label[j]==0):
            FalseNegative+=1
        elif (output[j]<0.5) and (label[j]==0):
            TruePositive+=1
        elif (output[j]<0.5) and (label[j]==1):
            FalsePositive+=1

    return TrueNegative,FalseNegative,TruePositive,FalsePositive

In [11]:
train_on_gpu = torch.cuda.is_available()
train_on_gpu

if train_on_gpu:
    model.cuda()

In [12]:
TN=0
FN=0
TP=1
FP=0
#del img0, img1, label
print("Testing...")
bar = tq(test_dataset_loader, postfix={"Accuracy":0.0})
for img0, img1, label in bar:
    
    if train_on_gpu:
        img0, img1, label = img0.cuda(), img1.cuda(), label.cuda()
    
    output = model(img0, img1)
    
    total = label.size(0)
    TN,FN,TP,FP = correct_out(output,label,TrueNegative=TN,FalseNegative=FN,TruePositive=TP,FalsePositive=FP)

        
    Pr = TP / (TP + FP)
    Rec = TP /(TP + FN)
    Acc = (TP+TN)/(TP + TN + FN + FP)
    Jaccard = TP/(TP + FN + FP) 
    bar.set_postfix(ordered_dict={"Accuracy":(Acc)*100})  

print('{} correct predictions out of {}\nAccuracy : {:.2f}\nJaccard : {:.2f}'.format((TP+TN),(TP+TN+FN+FN), (Acc)*100,Jaccard*100))

Testing...


  0%|          | 0/4115 [00:00<?, ?it/s, Accuracy=0]